# Testing for E1 - using all spindles, training with all data
fistly define the UUID of the experiment to use

In [1]:
experimentId="953846a1-fd19-4195-904d-79525add579b"

-------------------------------------------------------

In [2]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import sklearn.metrics as skm

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"\\COGNITION"
dreamspath=datapath+"\\DREAMS"
masspath=datapath+"\\MASS"


## load mass

In [3]:
samplerate=200  #Should rethink this

In [4]:
annotations, signalsMetadata = loadMASSSpindles(masspath,forceSamplerate=samplerate)

In [5]:
#consider to include this in a function
minDuration=0.3
maxDuration=5
annotations=annotations[annotations.type=='spindle']
annotations=annotations[annotations.duration>minDuration]
annotations=annotations[annotations.duration<maxDuration]
annotations=annotations.reset_index(drop=True)

In [6]:
#check with EDA results (not needed)
print(1-len(annotations)/33458)

0.005648873214178951


In [7]:
annotations.head()

,type,expert,subjectId,labelerId,startTime,duration,samplerate,stopTime,startInd,stopInd
0,spindle,E1,0001,0001,888.327805,0.640579,200,888.968384,177666,177794
1,spindle,E1,0001,0001,905.758061,0.578094,200,906.336155,181152,181267
2,spindle,E1,0001,0001,917.731574,0.847603,200,918.579177,183546,183716
3,spindle,E1,0001,0001,922.078189,0.878845,200,922.957034,184416,184591
4,spindle,E1,0001,0001,939.055445,0.757767,200,939.813212,187811,187963


In [8]:
signalsMetadata.head()

,subjectId,file,channel,duration,samplerate,isOriginalSamplerate,database
0,0001,MASS_0001.pkl,C3-CLE,28956.0,200,False,MASS
1,0002,MASS_0002.pkl,C3-CLE,35016.0,200,False,MASS
2,0003,MASS_0003.pkl,C3-CLE,36760.0,200,False,MASS
3,0004,MASS_0004.pkl,C3-CLE,28004.0,200,False,MASS
4,0005,MASS_0005.pkl,C3-CLE,31244.0,200,False,MASS


## Load experiment results

In [9]:
experimentModels, featureSelection = loadExperiment(experimentId,datapath)

In [10]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0001, 0005, 0006, 0009, 0010, 0011, 0012, 001...","[0018, 0017, 0007]","[0013, 0002, 0003]",5c1fbad9-f02b-46f7-8e51-59ddaf0cf4eb,0.017868
1,0001,E1,[0001],"[0002, 0003, 0006, 0007, 0009, 0013, 0017, 001...","[0011, 0001, 0010]","[0005, 0014, 0012]",f3f5e58f-0068-4046-b0a7-a62694b1fc26,0.017785
2,0001,E1,[0001],"[0003, 0005, 0007, 0009, 0010, 0012, 0013, 001...","[0001, 0002, 0019]","[0011, 0006, 0018]",919e22eb-bc1e-4601-a070-09b25fe7b52f,0.017121
3,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007, 0012, 0013, 001...","[0009, 0011, 0017]","[0019, 0001, 0010]",c2e2175e-cc0c-4536-86b5-c7a6ef28b104,0.017512
4,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0010, 0011, 0013, 001...","[0014, 0012, 0002]","[0009, 0017, 0007]",0f1d8cd5-f3dc-4419-a46b-69852cdbdadf,0.016249
5,0002,E2,[0002],"[0001, 0005, 0006, 0009, 0010, 0011, 0012, 001...","[0018, 0017, 0007]","[0013, 0002, 0003]",707edb30-2665-4c58-a31e-dbcc539eb169,0.061583
6,0002,E2,[0002],"[0002, 0003, 0006, 0007, 0009, 0013, 0017, 001...","[0011, 0001, 0010]","[0005, 0014, 0012]",b302a855-d3de-463c-b23c-53ddf80cbe46,0.053240
7,0002,E2,[0002],"[0003, 0005, 0007, 0009, 0010, 0012, 0013, 001...","[0001, 0002, 0019]","[0011, 0006, 0018]",cca39540-d753-4b1f-b2f8-6f3e209a0deb,0.056399
8,0002,E2,[0002],"[0002, 0003, 0005, 0006, 0007, 0012, 0013, 001...","[0009, 0011, 0017]","[0019, 0001, 0010]",4c8b7412-a382-4f84-b66c-acdcca60a4a9,0.051154
9,0002,E2,[0002],"[0001, 0003, 0005, 0006, 0010, 0011, 0013, 001...","[0014, 0012, 0002]","[0009, 0017, 0007]",cd965d0c-da5d-44f4-b337-5d6385102e27,0.053971


In [11]:
#we show the difference in class inbalance for the annotation criteria considered
experimentModels[['criteriumName','spindleTimeRate']].groupby('criteriumName').describe()

spindleTimeRate                                          \
                        count      mean       std       min       25%   
criteriumName                                                           
E1                        5.0  0.017307  0.000659  0.016249  0.017121   
E2                        5.0  0.055269  0.003996  0.051154  0.053240   

                                             
                    50%       75%       max  
criteriumName                                
E1             0.017512  0.017785  0.017868  
E2             0.053971  0.056399  0.061583

In [12]:
featureSelection

,window,characteristic,bandName,score
0,2,hjortActivity,beta2,69.0
1,1,hjortActivity,sigma,63.0
2,2,hjortActivity,beta1,42.0
3,2,hjortActivity,delta2,41.0
4,1.5,sigmaIndex,broadband,39.0
5,2,sigmaIndex,broadband,39.0
6,2,hjortActivity,theta,38.0
7,2,hjortMobility,beta1,37.0
8,2,hjortActivity,alpha,35.0
9,1,sigmaIndex,broadband,34.0


## Hyperparameter definition
this should come from a previous evaluation notebook

In [13]:
hyperClose=0.1
hyperDuration=minDuration
hyperThres=0.3
hyperDepth=30

## Testing with E1 criterium
we test the optimal points for the prediction threshold and number of boost iterations in the different validation groups

In [14]:
experimentModels=experimentModels[experimentModels.criteriumName=='E1'].reset_index(drop=True)
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0001, 0005, 0006, 0009, 0010, 0011, 0012, 001...","[0018, 0017, 0007]","[0013, 0002, 0003]",5c1fbad9-f02b-46f7-8e51-59ddaf0cf4eb,0.017868
1,0001,E1,[0001],"[0002, 0003, 0006, 0007, 0009, 0013, 0017, 001...","[0011, 0001, 0010]","[0005, 0014, 0012]",f3f5e58f-0068-4046-b0a7-a62694b1fc26,0.017785
2,0001,E1,[0001],"[0003, 0005, 0007, 0009, 0010, 0012, 0013, 001...","[0001, 0002, 0019]","[0011, 0006, 0018]",919e22eb-bc1e-4601-a070-09b25fe7b52f,0.017121
3,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007, 0012, 0013, 001...","[0009, 0011, 0017]","[0019, 0001, 0010]",c2e2175e-cc0c-4536-86b5-c7a6ef28b104,0.017512
4,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0010, 0011, 0013, 001...","[0014, 0012, 0002]","[0009, 0017, 0007]",0f1d8cd5-f3dc-4419-a46b-69852cdbdadf,0.016249


In [15]:
f1s=[]
precisions=[]
recalls=[]

eventF1s=[]
eventPrecisions=[]
eventRecalls=[]

for ind,row in experimentModels.iterrows():
    print('*************************')
    print(str(ind+1)+' of '+str(len(experimentModels)) )
    #load model
    model=loadBooster(row.modelId,experimentId,datapath)

    #Define annotations criterium
    usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)

    thisF1s=[]
    thisPs=[]
    thisRs=[]

    thisEF1s=[]
    thisEPs=[]
    thisERs=[]
    
    for testSubjectId in row.test:

        #Load features and labels
        testFeatures=loadFeatureMatrix([testSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
        testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

        #Predict
        testDMatrix=xgb.DMatrix(data=testFeatures)
        probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
        rawLabels=probabilities>=hyperThres
       
        #Process
        processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
        #Processed metrics
        tp=np.sum(processedLabels*testLabels)
        fp=np.sum(processedLabels*(1-testLabels))
        tn=np.sum((1-processedLabels)*(1-testLabels))
        fn=np.sum((1-processedLabels)*testLabels)
        #Processed appends
        thisF1s.append(2*tp/(2*tp+fp+fn))
        thisPs.append(tp/(tp+fp))
        thisRs.append(tp/(tp+fn))

        #By-event metrics
        processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
        processedAnnotations=processedAnnotations[processedAnnotations.duration<maxDuration]
        gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
        f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations)
    
        #calculate metrics
        thisEF1s.append(f)
        thisEPs.append(p)
        thisERs.append(r)
    
    f1s.append(np.mean(thisF1s))
    precisions.append(np.mean(thisPs))
    recalls.append(np.mean(thisRs))

    eventF1s.append(np.mean(thisEF1s))
    eventPrecisions.append(np.mean(thisEPs))
    eventRecalls.append(np.mean(thisERs))
    
#include metrics in the dataframe
experimentModels['f1']=f1s
experimentModels['precision']=precisions
experimentModels['recall']=recalls

experimentModels['eventF1']=eventF1s
experimentModels['eventPrecision']=eventPrecisions
experimentModels['eventRecall']=eventRecalls


*************************
1 of 5


*************************
2 of 5
*************************
3 of 5
*************************
4 of 5
*************************
5 of 5


In [16]:
dumpPickle('experimentModelsTest_E1_temp.pkl',experimentModels)

In [17]:
experimentModels.columns

Index(['criteriumId', 'criteriumName', 'labelerIdList', 'train', 'val', 'test',
       'modelId', 'spindleTimeRate', 'f1', 'precision', 'recall', 'eventF1',
       'eventPrecision', 'eventRecall'],
      dtype='object')

In [18]:
fig=px.scatter(experimentModels,x='eventF1',y='eventPrecision',color='modelId',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [19]:
fig=px.scatter(experimentModels,x='eventF1',y='eventRecall',color='modelId',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [20]:
experimentModels[['modelId','eventF1','eventPrecision','eventRecall']].groupby('modelId').describe(percentiles=[0.5])

eventF1                          \
                                       count      mean std       min   
modelId                                                                
0f1d8cd5-f3dc-4419-a46b-69852cdbdadf     1.0  0.706121 NaN  0.706121   
5c1fbad9-f02b-46f7-8e51-59ddaf0cf4eb     1.0  0.713508 NaN  0.713508   
919e22eb-bc1e-4601-a070-09b25fe7b52f     1.0  0.710848 NaN  0.710848   
c2e2175e-cc0c-4536-86b5-c7a6ef28b104     1.0  0.629340 NaN  0.629340   
f3f5e58f-0068-4046-b0a7-a62694b1fc26     1.0  0.674704 NaN  0.674704   

                                                         eventPrecision  \
                                           50%       max          count   
modelId                                                                   
0f1d8cd5-f3dc-4419-a46b-69852cdbdadf  0.706121  0.706121            1.0   
5c1fbad9-f02b-46f7-8e51-59ddaf0cf4eb  0.713508  0.713508            1.0   
919e22eb-bc1e-4601-a070-09b25fe7b52f  0.710848  0.710848            1.0   
c2e2175e-cc0c-4536-86b5-c7a6ef28b104  0.629340  0.629340            1.0   
f3f5e58f-0068-4046-b0a7-a62694b1fc26  0.674704  0.674704            1.0   

                                                                        \
                                          mean std       min       50%   
modelId                                                                  
0f1d8cd5-f3dc-4419-a46b-69852cdbdadf  0.732611 NaN  0.732611  0.732611   
5c1fbad9-f02b-46f7-8e51-59ddaf0cf4eb  0.644596 NaN  0.644596  0.644596   
919e22eb-bc1e-4601-a070-09b25fe7b52f  0.690502 NaN  0.690502  0.690502   
c2e2175e-cc0c-4536-86b5-c7a6ef28b104  0.688244 NaN  0.688244  0.688244   
f3f5e58f-0068-4046-b0a7-a62694b1fc26  0.566676 NaN  0.566676  0.566676   

                                               eventRecall                \
                                           max       count      mean std   
modelId                                                                    
0f1d8cd5-f3dc-4419-a46b-69852cdbdadf  0.732611         1.0  0.723078 NaN   
5c1fbad9-f02b-46f7-8e51-59ddaf0cf4eb  0.644596         1.0  0.816690 NaN   
919e22eb-bc1e-4601-a070-09b25fe7b52f  0.690502         1.0  0.770473 NaN   
c2e2175e-cc0c-4536-86b5-c7a6ef28b104  0.688244         1.0  0.730411 NaN   
f3f5e58f-0068-4046-b0a7-a62694b1fc26  0.566676         1.0  0.868199 NaN   

                                                                    
                                           min       50%       max  
modelId                                                             
0f1d8cd5-f3dc-4419-a46b-69852cdbdadf  0.723078  0.723078  0.723078  
5c1fbad9-f02b-46f7-8e51-59ddaf0cf4eb  0.816690  0.816690  0.816690  
919e22eb-bc1e-4601-a070-09b25fe7b52f  0.770473  0.770473  0.770473  
c2e2175e-cc0c-4536-86b5-c7a6ef28b104  0.730411  0.730411  0.730411  
f3f5e58f-0068-4046-b0a7-a62694b1fc26  0.868199  0.868199  0.868199

In [21]:
experimentModels[['modelId','eventF1','eventPrecision','eventRecall']].groupby('modelId',as_index=False).mean()

,modelId,eventF1,eventPrecision,eventRecall
0,0f1d8cd5-f3dc-4419-a46b-69852cdbdadf,0.706121,0.732611,0.723078
1,5c1fbad9-f02b-46f7-8e51-59ddaf0cf4eb,0.713508,0.644596,0.816690
2,919e22eb-bc1e-4601-a070-09b25fe7b52f,0.710848,0.690502,0.770473
3,c2e2175e-cc0c-4536-86b5-c7a6ef28b104,0.629340,0.688244,0.730411
4,f3f5e58f-0068-4046-b0a7-a62694b1fc26,0.674704,0.566676,0.868199


In [22]:
experimentModels[['modelId','eventF1','eventPrecision','eventRecall']].groupby('modelId',as_index=False).mean().mean()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_9992\3541183823.py:1: FutureWarning:

The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.686904
eventPrecision    0.664526
eventRecall       0.781770
dtype: float64

In [23]:
experimentModels[['modelId','eventF1','eventPrecision','eventRecall']].groupby('modelId',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_9992\376514559.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.035752
eventPrecision    0.062938
eventRecall       0.061032
dtype: float64

In [24]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels.eventPrecision,
    'event F1':experimentModels.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels.eventRecall,
    'event F1':experimentModels.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [25]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

## Restrict to only N2 stage

In [26]:
experimentModels_N2=experimentModels.copy()
experimentModels_N2.head()

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate,f1,precision,recall,eventF1,eventPrecision,eventRecall
0,0001,E1,[0001],"[0001, 0005, 0006, 0009, 0010, 0011, 0012, 001...","[0018, 0017, 0007]","[0013, 0002, 0003]",5c1fbad9-f02b-46f7-8e51-59ddaf0cf4eb,0.017868,0.667482,0.628758,0.728434,0.713508,0.644596,0.816690
1,0001,E1,[0001],"[0002, 0003, 0006, 0007, 0009, 0013, 0017, 001...","[0011, 0001, 0010]","[0005, 0014, 0012]",f3f5e58f-0068-4046-b0a7-a62694b1fc26,0.017785,0.618007,0.527793,0.792828,0.674704,0.566676,0.868199
2,0001,E1,[0001],"[0003, 0005, 0007, 0009, 0010, 0012, 0013, 001...","[0001, 0002, 0019]","[0011, 0006, 0018]",919e22eb-bc1e-4601-a070-09b25fe7b52f,0.017121,0.647533,0.674013,0.657270,0.710848,0.690502,0.770473
3,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007, 0012, 0013, 001...","[0009, 0011, 0017]","[0019, 0001, 0010]",c2e2175e-cc0c-4536-86b5-c7a6ef28b104,0.017512,0.580825,0.674347,0.650820,0.629340,0.688244,0.730411
4,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0010, 0011, 0013, 001...","[0014, 0012, 0002]","[0009, 0017, 0007]",0f1d8cd5-f3dc-4419-a46b-69852cdbdadf,0.016249,0.631782,0.703761,0.617388,0.706121,0.732611,0.723078


In [27]:
annotationsN2=pd.read_csv(datapath+'/MASS/annotations/spindlesFilteredN2.csv')
annotationsN2['samplerate']=samplerate
annotationsN2['subjectId']=annotationsN2.apply(
    lambda row: str(row.subjectId).zfill(4),axis=1)
annotationsN2['labelerId']=annotationsN2.apply(
    lambda row: str(row.labelerId).zfill(4),axis=1)
annotationsN2['stopTime']=annotationsN2.apply(
    lambda row: row.startTime+row.duration , axis=1)
annotationsN2['startInd']=annotationsN2.apply(
    lambda row: seconds2index(row.startTime,row.samplerate) , axis=1)
annotationsN2['stopInd']=annotationsN2.apply(
    lambda row: seconds2index(row.stopTime,row.samplerate) , axis=1)

In [28]:
## load hypnograms
stagesAnnotations=pd.read_csv(datapath+"\MASS\stages\stages.csv")
stagesAnnotations['samplerate']=samplerate
stagesAnnotations['subjectId']=stagesAnnotations.apply(
    lambda row: str(row.subjectId).zfill(4),axis=1)
stagesAnnotations['stopTime']=stagesAnnotations.apply(
    lambda row: row.startTime+row.duration , axis=1)
stagesAnnotations['startInd']=stagesAnnotations.apply(
    lambda row: seconds2index(row.startTime,row.samplerate) , axis=1)
stagesAnnotations['stopInd']=stagesAnnotations.apply(
    lambda row: seconds2index(row.stopTime,row.samplerate) , axis=1)
stagesAnnotations.head(5)

hypnograms={}
for ind, row in signalsMetadata.iterrows():
    subjectId=row.subjectId
    thisStages=stagesAnnotations[stagesAnnotations.subjectId==subjectId]
    excerptDimension=int(row.duration*row.samplerate)
    thisHypnogram=np.ones((excerptDimension,))*np.nan
    for ind_stg, row_stg in thisStages.iterrows():
        thisHypnogram[row_stg.startInd:row_stg.stopInd]=row_stg.value
    hypnograms[subjectId]=thisHypnogram   

In [29]:
f1s=[]
precisions=[]
recalls=[]

eventF1s=[]
eventPrecisions=[]
eventRecalls=[]

for ind,row in experimentModels_N2.iterrows():
    print('*************************')
    print(str(ind+1)+' of '+str(len(experimentModels_N2)) )
    #load model
    model=loadBooster(row.modelId,experimentId,datapath)

    #Define annotations criterium
    usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)

    thisF1s=[]
    thisPs=[]
    thisRs=[]

    thisEF1s=[]
    thisEPs=[]
    thisERs=[]
    
    for testSubjectId in row.test:
        testHypnogram=hypnograms[testSubjectId]
        testN2Mask=testHypnogram==2

        #Load features and labels
        testFeatures=loadFeatureMatrix([testSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
        testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

        #Predict
        testDMatrix=xgb.DMatrix(data=testFeatures)
        probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
        rawLabels=probabilities>=hyperThres

        #Apply mask ---------------------------------->
        rawLabels=rawLabels*testN2Mask
        #<---------------------------------------------
       
        #Process
        processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
        #Processed metrics
        tp=np.sum(processedLabels*testLabels)
        fp=np.sum(processedLabels*(1-testLabels))
        tn=np.sum((1-processedLabels)*(1-testLabels))
        fn=np.sum((1-processedLabels)*testLabels)
        #Processed appends
        thisF1s.append(2*tp/(2*tp+fp+fn))
        thisPs.append(tp/(tp+fp))
        thisRs.append(tp/(tp+fn))

        #By-event metrics
        processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
        processedAnnotations=processedAnnotations[processedAnnotations.duration<maxDuration]
        gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
        f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations)
    
        #calculate metrics
        thisEF1s.append(f)
        thisEPs.append(p)
        thisERs.append(r)
    
    f1s.append(np.mean(thisF1s))
    precisions.append(np.mean(thisPs))
    recalls.append(np.mean(thisRs))

    eventF1s.append(np.mean(thisEF1s))
    eventPrecisions.append(np.mean(thisEPs))
    eventRecalls.append(np.mean(thisERs))
    
#include metrics in the dataframe
experimentModels_N2['f1']=f1s
experimentModels_N2['precision']=precisions
experimentModels_N2['recall']=recalls

experimentModels_N2['eventF1']=eventF1s
experimentModels_N2['eventPrecision']=eventPrecisions
experimentModels_N2['eventRecall']=eventRecalls


*************************
1 of 5
*************************
2 of 5
*************************
3 of 5
*************************
4 of 5
*************************
5 of 5


In [31]:
dumpPickle('experimentModelsTest_N2_E1_temp.pkl',experimentModels_N2)

In [35]:
fig=px.scatter(experimentModels_N2,x='eventF1',y='eventPrecision',color='modelId',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['eventF1'], y=experimentModels_N2['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [36]:
fig=px.scatter(experimentModels_N2,x='eventF1',y='eventRecall',color='modelId',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['eventF1'], y=experimentModels_N2['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [38]:
experimentModels_N2[['modelId','eventF1','eventPrecision','eventRecall']].groupby('modelId').describe(percentiles=[0.5])

eventF1                          \
                                       count      mean std       min   
modelId                                                                
0f1d8cd5-f3dc-4419-a46b-69852cdbdadf     1.0  0.742430 NaN  0.742430   
5c1fbad9-f02b-46f7-8e51-59ddaf0cf4eb     1.0  0.748374 NaN  0.748374   
919e22eb-bc1e-4601-a070-09b25fe7b52f     1.0  0.735098 NaN  0.735098   
c2e2175e-cc0c-4536-86b5-c7a6ef28b104     1.0  0.654319 NaN  0.654319   
f3f5e58f-0068-4046-b0a7-a62694b1fc26     1.0  0.706986 NaN  0.706986   

                                                         eventPrecision  \
                                           50%       max          count   
modelId                                                                   
0f1d8cd5-f3dc-4419-a46b-69852cdbdadf  0.742430  0.742430            1.0   
5c1fbad9-f02b-46f7-8e51-59ddaf0cf4eb  0.748374  0.748374            1.0   
919e22eb-bc1e-4601-a070-09b25fe7b52f  0.735098  0.735098            1.0   
c2e2175e-cc0c-4536-86b5-c7a6ef28b104  0.654319  0.654319            1.0   
f3f5e58f-0068-4046-b0a7-a62694b1fc26  0.706986  0.706986            1.0   

                                                                        \
                                          mean std       min       50%   
modelId                                                                  
0f1d8cd5-f3dc-4419-a46b-69852cdbdadf  0.792676 NaN  0.792676  0.792676   
5c1fbad9-f02b-46f7-8e51-59ddaf0cf4eb  0.701707 NaN  0.701707  0.701707   
919e22eb-bc1e-4601-a070-09b25fe7b52f  0.736158 NaN  0.736158  0.736158   
c2e2175e-cc0c-4536-86b5-c7a6ef28b104  0.738444 NaN  0.738444  0.738444   
f3f5e58f-0068-4046-b0a7-a62694b1fc26  0.620248 NaN  0.620248  0.620248   

                                               eventRecall                \
                                           max       count      mean std   
modelId                                                                    
0f1d8cd5-f3dc-4419-a46b-69852cdbdadf  0.792676         1.0  0.723078 NaN   
5c1fbad9-f02b-46f7-8e51-59ddaf0cf4eb  0.701707         1.0  0.816690 NaN   
919e22eb-bc1e-4601-a070-09b25fe7b52f  0.736158         1.0  0.770473 NaN   
c2e2175e-cc0c-4536-86b5-c7a6ef28b104  0.738444         1.0  0.730411 NaN   
f3f5e58f-0068-4046-b0a7-a62694b1fc26  0.620248         1.0  0.867258 NaN   

                                                                    
                                           min       50%       max  
modelId                                                             
0f1d8cd5-f3dc-4419-a46b-69852cdbdadf  0.723078  0.723078  0.723078  
5c1fbad9-f02b-46f7-8e51-59ddaf0cf4eb  0.816690  0.816690  0.816690  
919e22eb-bc1e-4601-a070-09b25fe7b52f  0.770473  0.770473  0.770473  
c2e2175e-cc0c-4536-86b5-c7a6ef28b104  0.730411  0.730411  0.730411  
f3f5e58f-0068-4046-b0a7-a62694b1fc26  0.867258  0.867258  0.867258

In [39]:
experimentModels_N2[['modelId','eventF1','eventPrecision','eventRecall']].groupby('modelId',as_index=False).mean()

,modelId,eventF1,eventPrecision,eventRecall
0,0f1d8cd5-f3dc-4419-a46b-69852cdbdadf,0.742430,0.792676,0.723078
1,5c1fbad9-f02b-46f7-8e51-59ddaf0cf4eb,0.748374,0.701707,0.816690
2,919e22eb-bc1e-4601-a070-09b25fe7b52f,0.735098,0.736158,0.770473
3,c2e2175e-cc0c-4536-86b5-c7a6ef28b104,0.654319,0.738444,0.730411
4,f3f5e58f-0068-4046-b0a7-a62694b1fc26,0.706986,0.620248,0.867258


In [40]:
experimentModels_N2[['modelId','eventF1','eventPrecision','eventRecall']].groupby('modelId',as_index=False).mean().mean()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_9992\3291470165.py:1: FutureWarning:

The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.717441
eventPrecision    0.717847
eventRecall       0.781582
dtype: float64

In [41]:
experimentModels_N2[['modelId','eventF1','eventPrecision','eventRecall']].groupby('modelId',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_9992\2063147112.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.038687
eventPrecision    0.063532
eventRecall       0.060699
dtype: float64

In [42]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels_N2.eventPrecision,
    'event F1':experimentModels_N2.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels_N2.eventRecall,
    'event F1':experimentModels_N2.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [43]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [44]:
annotationPairToGraph(gtAnnotations,processedAnnotations)